In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import joblib
import glob  # for file matching

# List all CSV files you want to load (adjust path/pattern)
file_paths = glob.glob(r"C:\Users\debas\AIRTRAJ\dataset-research\Data Preprocessing\Oslo\processed_data\*.csv")

# Load and concatenate all files
df_list = []
for file in file_paths:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

# Combine all data into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

# Use only these 4 features for input & prediction
features = ['lat', 'lon', 'alt', 'time']
data = df[features]

# Scale all features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Save the scaler for later use
joblib.dump(scaler, "coord_scaler.pkl")

# Create sequences function (unchanged)
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])  # Next timestep
    return np.array(X), np.array(y)

SEQ_LENGTH = 10
X, y = create_sequences(data_scaled, SEQ_LENGTH)

# Split data (no shuffle to preserve time order)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(SEQ_LENGTH, len(features)), activation='tanh'),
    Dense(4)  # Output: lat, lon, alt, time
])

model.compile(optimizer='adam', loss='mse')

# Train
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.1)

# Save model using new .keras format
model.save("coord_predictor.keras")


c:\Users\debas\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
93139/93139 ━━━━━━━━━━━━━━━━━━━━ 823s 9ms/step - loss: 0.0103 - val_loss: 0.0130
Epoch 2/3
93139/93139 ━━━━━━━━━━━━━━━━━━━━ 594s 6ms/step - loss: 0.0094 - val_loss: 0.0112
Epoch 3/3
93139/93139 ━━━━━━━━━━━━━━━━━━━━ 577s 6ms/step - loss: 0.0090 - val_loss: 0.0103
